In [2]:
import pyaudio
import os
os.chdir('../')
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError

# use this backend to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

ModuleNotFoundError: No module named 'pyaudio'

In [4]:
# create matplotlib figure and axes
fig, ax = plt.subplots(1, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)

# create a line object with random data
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
stream stopped
average frame rate = 21 FPS


##  Training Dataset Preparation

Input to model : 8 sec of audio clip with My sound Dubbed in between . 

Label : Label some number of time steps 

Inference : Model listens for 8 second for a while. Then, processes it . waits for a while. Then, Processes ..

More like, multithread, such that, by time it processes for prediction generation, it's preparing next wave .

The computaiton control is done by time.sleep .. 

In [1]:
import numpy as np
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import IPython
%matplotlib inline

ModuleNotFoundError: No module named 'pydub'

In [ ]:
import logging

In [ ]:
os.chdir('../')

In [ ]:
# IPython.display.Audio("data/external/sample_internet.wav")

In [ ]:
from pydub import AudioSegment
song = AudioSegment.from_wav('./data/external/podact_data/train_1.wav')

In [ ]:
type(song)

In [ ]:
## load training examles

662

In [ ]:
root_ =  './data/external/podact_data/activates_robin'
clips = []
for i in os.listdir(root_):
    if i[0].isdigit():
        clip_path = os.path.join( root_ , i ) 
        clip = AudioSegment.from_wav(clip_path)
        print('Traning Clip Vol : ' , clip.dBFS )
        print('Traning Clip Len in Mills :' , len(clip))
        clips.append( clip )

In [ ]:
play( AudioSegment.from_wav(clip_path)  )

In [ ]:
total_song_length = song.duration_seconds

In [ ]:
from pydub.playback import play
import random

In [24]:
n = 1000
play( song[23.9999999 * n : 29 * n] ) 

In [25]:
def sample_from_activate( clip_list ):
    return random.choice(clip_list)

## experimental 
def adjust_level(sound, deviation=None , default = True):
    if default or deviation is None:
        return sound
    else:
        difference = np.random.normal(clip.dBFS , deviation )
        print(f'Original : {clip.dBFS}')
        print(f'Deviated by : {difference}')
        return sound.apply_gain(difference)

def sample_from_background( background , total_duration_sec , clip_size = 8 ,  multiplier = 1000 ):
        
    total_duration_ = total_duration_sec - clip_size
    selection_start = np.random.uniform(  0 , total_duration_  ) 
    selection_end = (selection_start + clip_size) * multiplier
        
    background_clip = background[ selection_start * multiplier : selection_end  ] 
    return background_clip

def overlay_clip_to_bg( background , clip ):
    '''
        Dub Clip to background
    '''
    overlay_point_start = np.random.uniform( 0 , background.duration_seconds ) * 1000
    overlay_point_end = overlay_point_start + len(clip)
    overlay_point_end = min( len(background) , overlay_point_end  )
    
    return background.overlay( clip , position =  overlay_point_start ) , overlay_point_start , overlay_point_end

In [26]:
len(clip)

662

In [27]:
%%time
## check the goddamn speed
for i in range(3000):
    _ = sample_from_background(song, total_song_length)

Wall time: 1.84 s


In [28]:
def generate_single_example( background , voice_clips ):
    
    total_background_length = background.duration_seconds 
    background = sample_from_background( background , total_background_length )
    
    ## positive labels here
    if np.random.uniform(0,1) > 0.5:
        print('Positive Example Added')
        clip_to_dub = sample_from_activate( voice_clips )
        overlayed_clip, time_stamp_start , time_stamp_end = overlay_clip_to_bg( background, clip_to_dub )  
        return overlayed_clip , time_stamp_start , time_stamp_end
    
    else:
        print('Negative Example Added')
        return background , None, None

In [37]:
test_overlay , s ,e = generate_single_example( song , clips )

Positive Example Added


In [38]:
s,e

(1786.1245853711428, 2738.1245853711425)

In [39]:
len(test_overlay)

8000

In [40]:
play(test_overlay)

In [41]:
pwd

'c:\\Users\\Robin\\Downloads\\million_dollar_projects\\podcast_research\\Podcast-Audio-Processing'

In [42]:
test_overlay.export('data\\external\\processed\\test.wav',format='wav')

<_io.BufferedRandom name='data\\external\\processed\\test.wav'>

In [43]:
import pandas as pd
meta_data = pd.DataFrame(columns=['filename','start_time','end_time'])

In [44]:
def prep_examples( no_of_examples = 2000 ):

    save_path = "data\\external\\processed\\"
    import pandas as pd

    meta_data = pd.DataFrame(columns=['filename','start_time','end_time'])
    
    for i in range( no_of_examples ):
        save_name = f'{i}.wav'
        sample , s ,e = generate_single_example( song , clips )
        sample.export( os.path.join( save_path , save_name ) ,format='wav')

        meta_data.loc[i] = [ save_name , s , e   ] 
    
    return meta_data


In [45]:
meta = prep_examples(2000)

xample Added
Negative Example Added
Negative Example Added
Positive Example Added
Positive Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Negative Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Negative Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Positive Example Added
Positive Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Positive Example Added
Positive Example Added
Positive Example Added
Negative Example Added
Negative Example Added
Positive Example Adde

In [46]:
meta.start_time.isna().value_counts()

False    1043
True      957
Name: start_time, dtype: int64

In [47]:
meta.to_csv('data//meta_data.csv',index=False)

In [51]:
for i in os.listdir("data\external\processed"):
    if i[0].isdigit():

        


['0.wav',
 '1.wav',
 '10.wav',
 '100.wav',
 '1000.wav',
 '1001.wav',
 '1002.wav',
 '1003.wav',
 '1004.wav',
 '1005.wav',
 '1006.wav',
 '1007.wav',
 '1008.wav',
 '1009.wav',
 '101.wav',
 '1010.wav',
 '1011.wav',
 '1012.wav',
 '1013.wav',
 '1014.wav',
 '1015.wav',
 '1016.wav',
 '1017.wav',
 '1018.wav',
 '1019.wav',
 '102.wav',
 '1020.wav',
 '1021.wav',
 '1022.wav',
 '1023.wav',
 '1024.wav',
 '1025.wav',
 '1026.wav',
 '1027.wav',
 '1028.wav',
 '1029.wav',
 '103.wav',
 '1030.wav',
 '1031.wav',
 '1032.wav',
 '1033.wav',
 '1034.wav',
 '1035.wav',
 '1036.wav',
 '1037.wav',
 '1038.wav',
 '1039.wav',
 '104.wav',
 '1040.wav',
 '1041.wav',
 '1042.wav',
 '1043.wav',
 '1044.wav',
 '1045.wav',
 '1046.wav',
 '1047.wav',
 '1048.wav',
 '1049.wav',
 '105.wav',
 '1050.wav',
 '1051.wav',
 '1052.wav',
 '1053.wav',
 '1054.wav',
 '1055.wav',
 '1056.wav',
 '1057.wav',
 '1058.wav',
 '1059.wav',
 '106.wav',
 '1060.wav',
 '1061.wav',
 '1062.wav',
 '1063.wav',
 '1064.wav',
 '1065.wav',
 '1066.wav',
 '1067.wav',
 

In [54]:
int('3')

3